In [ ]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
from quantumnet.objects import Request
import asyncio
path = "resultados/reativo/async.csv"
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 1000
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

In [ ]:
async def process_request(request, rede, controlador, current_time, execution_delay=3):
    """
    Processa uma única request após um delay de registro (3 time-slots).
    Se a regra já existe, executa diretamente.
    """
    request.starttime = current_time
    alice = rede.get_host(request.alice)
    rule = alice.find_rule_by_request(request)
    
    # Caso não exista um match na tabela
    if rule is False:
        print(f"[Time {current_time}] Registrando nova regra para Request {request}.")
        controlador.add_match_route_rule_in_host_reactive(request)
        await asyncio.sleep(execution_delay)  # Delay para registro
        print(f"[Time {current_time + execution_delay}] Regra registrada para {request}, executando:")
        rule = alice.find_rule_by_request(request)
        controlador.run_rule(rule[1])
        request.endtime = current_time + execution_delay
        
        # Registra no CSV como um novo registro
        register_request(request, "Novo Registro", path)
    # Caso a regra já exista
    else:
        await asyncio.sleep(1)
        print(f"[Time {current_time}] Executando regra para Request {request}:")
        controlador.run_rule(rule[1])
        request.endtime = current_time + 1
        
        # Registra no CSV como já registrado
        register_request(request, "Já Registrado", path)
    
    print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

async def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com timeslots contínuos usando programação assíncrona.
    - As requests são processadas simultaneamente.
    - O tempo avança de forma contínua enquanto as requisições são executadas.
    """
    current_time = 0  # Tempo global inicial
    tasks = []  # Lista de tarefas assíncronas para execução das requests

    while requests or tasks:
        # Adicionar nova request ao processamento
        if requests:
            request = requests.pop(0)
            print(f"[Time {current_time}] Request {request} recebida.")
            task = asyncio.create_task(process_request(request, rede, controlador, current_time))
            tasks.append(task)

        # Aguarda 1 time-slot antes de avançar o tempo
        await asyncio.sleep(1)
        current_time += 1

        # Filtrar tarefas concluídas
        tasks = [task for task in tasks if not task.done()]

    print("Simulação concluída!")

# Uso da função
# asyncio.run(simulate_requests_with_timeslots(rede, requests, controlador))
r1 = Request(0, 1, 0.5, 2)
r2 = Request(0, 1, 0.5, 2)
r3 = Request(0, 1, 0.5, 2)
r4 = Request(0, 1, 0.5, 2)
requests2 = [r1, r2, r3, r4]
await simulate_requests_with_timeslots(rede, requests, controlador)